In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
import torch
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow_text as text
import re
import string
tf.get_logger().setLevel('ERROR')

global_seed = 123
np.random.seed(global_seed)
tf.random.set_seed(global_seed)

c:\Users\xinhu\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Preprocess Dataset

In [2]:
df2 = pd.read_csv("C:/Clubs and Society/DevHack/DevHack/dataset/Resume/Resume.csv")
df2 = df2[['Resume_str', 'Category']]
df2.head(5)

,Resume_str,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,HR DIRECTOR Summary Over 2...,HR
3,HR SPECIALIST Summary Dedica...,HR
4,HR MANAGER Skill Highlights ...,HR


In [3]:
def preprocess_paragraph(paragraph):
    # Remove newlines
    paragraph = paragraph.replace('\n', ' ')

    # Remove connectors
    connectors = ['and', 'but', 'or', 'summary', 'experience', 'professional', 'skills', 'profile', 'education', 'skill', 'highlights']
    for connector in connectors:
        paragraph = re.sub(r'\b' + re.escape(connector) + r'\b', '', paragraph, flags=re.IGNORECASE)

    # Remove extra spaces
    paragraph = ' '.join(paragraph.split())

    return paragraph

In [4]:
df2['resume'] = df2['Resume_str'].apply(preprocess_paragraph)
df1 = df2[['resume', 'Category']]
df1.head(5)

,resume,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE HR ADMINI...,HR
1,"HR SPECIALIST, US HR OPERATIONS Versatile medi...",HR
2,"HR DIRECTOR Over 20 years in recruiting, 15 pl...",HR
3,"HR SPECIALIST Dedicated, Driven, Dynamic with ...",HR
4,HR MANAGER HR HR Department Startup Three New ...,HR


### Split Train, Test and Validation Data

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [6]:
class_mapping = {
    'HR': 0,
    'DESIGNER': 1,
    'INFORMATION-TECHNOLOGY': 2,
    'TEACHER': 3,
    'ADVOCATE': 4,
    'BUSINESS-DEVELOPMENT': 5,
    'HEALTHCARE': 6,
    'FITNESS': 7,
    'AGRICULTURE': 8,
    'BPO': 9,
    'SALES': 10,
    'CONSULTANT': 11,
    'DIGITAL-MEDIA': 12,
    'AUTOMOBILE': 13,
    'CHEF': 14,
    'FINANCE': 15,
    'APPAREL': 16,
    'ENGINEERING': 17,
    'ACCOUNTANT': 18,
    'CONSTRUCTION': 19,
    'PUBLIC-RELATIONS': 20,
    'BANKING': 21,
    'ARTS': 22,
    'AVIATION': 23
}

# Create a custom LabelEncoder with the provided mapping
label_encoder = LabelEncoder()
#label_encoder.classes_ = [class_mapping[label] for label in label_encoder.classes_]

In [7]:
df = df1 
X_train, x_test, y_train, y_test = train_test_split(df['resume'], df['Category'], test_size = 0.3, random_state = 42)
X_test, X_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size = 0.5, random_state = 42)

print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(y_train.shape))
print('X_test: ' + str(X_test.shape))
print('Y_test: ' + str(y_test.shape))
print('X_valid: ' + str(X_valid.shape))
print('Y_valid: ' + str(y_valid.shape))

X_train: (1738,)
Y_train: (1738,)
X_test: (373,)
Y_test: (373,)
X_valid: (373,)
Y_valid: (373,)


In [8]:
label_encoder = LabelEncoder()
Y_train = label_encoder.fit_transform(y_train.map(class_mapping))
Y_test = label_encoder.fit_transform(y_test.map(class_mapping))
Y_valid = label_encoder.fit_transform(y_valid.map(class_mapping))

## Loading models from TensorFlow Hub

In [9]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-2_H-128_A-2'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", 
                                                              #"bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", 
                                                              #"small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", 
                                                              #"small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


### Preprocessing Model

In [10]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [11]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_word_ids', 'input_type_ids', 'input_mask']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


### BERT Model

In [12]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [13]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1
Pooled Outputs Shape:(1, 128)
Pooled Outputs Values:[-0.9999946   0.14341967 -0.998978    0.9951447  -0.99974567  0.9131291
 -0.996158   -0.970975    0.09740017  0.01907059 -0.8488176  -0.09385555]
Sequence Outputs Shape:(1, 128, 128)
Sequence Outputs Values:[[-0.43723688 -1.0138297  -2.374474   ... -0.75897044 -2.0910316
  -0.25809968]
 [-1.3369719  -0.3688029   0.57868755 ... -2.1810174  -1.753092
  -0.09555102]
 [-1.0607055  -0.30371076  0.34461164 ... -1.2898594  -1.9519897
  -0.1223038 ]
 ...
 [-0.8602275  -0.54406166  0.6583735  ... -1.4765682  -1.7518609
   1.1197377 ]
 [-0.46869135 -0.5715234   0.67256564 ... -1.7302349  -1.9740777
   0.97479314]
 [ 0.02036636 -0.75511944  0.5948237  ... -1.9656786  -1.9051423
   0.48970205]]


### Model Construction

In [14]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)

    net = tf.keras.layers.Dense(256, activation='tanh')(net)
    net = tf.keras.layers.BatchNormalization()(net)
    net = tf.keras.layers.Dropout(0.3)(net)
    
    net = tf.keras.layers.Dense(128, activation='tanh')(net)
    net = tf.keras.layers.BatchNormalization()(net)
    net = tf.keras.layers.Dropout(0.3)(net)
    
    net = tf.keras.layers.Dense(24, activation='softmax', name='classifier')(net)
    
    return tf.keras.Model(text_input, net)

In [15]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor(
[[0.50997764 0.53745306 0.51133907 0.50612664 0.5033853  0.50569326
  0.5017349  0.50680566 0.5052632  0.50507325 0.51371014 0.50300115
  0.51127094 0.5434864  0.50803316 0.5023108  0.50513136 0.5133802
  0.51263547 0.5099814  0.5033673  0.5049385  0.5155581  0.51013786]], shape=(1, 24), dtype=float32)


### Model Training

In [16]:
#opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
classifier_model.compile(optimizer = opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Convert pandas Series to numpy arrays
X_train_np = np.array(X_train)
Y_train_np = np.array(Y_train)

# Convert validation data if needed
X_valid_np = np.array(X_valid)
Y_valid_np = np.array(Y_valid)

In [18]:
classifier_model.fit(x=X_train_np, y = Y_train_np, validation_data = (X_valid_np, Y_valid_np), epochs = 20)

Epoch 1/20
55/55 [==============================] - 25s 368ms/step - loss: 3.0135 - accuracy: 0.2509 - val_loss: 1.5964 - val_accuracy: 0.6220
Epoch 2/20
55/55 [==============================] - 21s 386ms/step - loss: 1.4781 - accuracy: 0.6335 - val_loss: 0.9779 - val_accuracy: 0.7614
Epoch 3/20
55/55 [==============================] - 23s 425ms/step - loss: 1.1715 - accuracy: 0.7221 - val_loss: 1.2507 - val_accuracy: 0.6863
Epoch 4/20
55/55 [==============================] - 25s 456ms/step - loss: 1.0186 - accuracy: 0.7514 - val_loss: 1.1786 - val_accuracy: 0.7105
Epoch 5/20
55/55 [==============================] - 27s 484ms/step - loss: 0.8528 - accuracy: 0.7883 - val_loss: 1.1868 - val_accuracy: 0.7399
Epoch 6/20
55/55 [==============================] - 29s 530ms/step - loss: 0.6853 - accuracy: 0.8222 - val_loss: 1.5809 - val_accuracy: 0.6971
Epoch 7/20
55/55 [==============================] - 27s 498ms/step - loss: 0.5782 - accuracy: 0.8516 - val_loss: 1.3169 - val_accuracy: 0.7426

In [19]:
X_test_np = np.array(X_test)
Y_test_np = np.array(Y_test)

classifier_model.evaluate(X_test_np, Y_test_np)

12/12 [==============================] - 2s 128ms/step - loss: 2.0136 - accuracy: 0.7346


[2.013561487197876, 0.7345844507217407]

In [20]:
classifier_model.save('./category_model')